In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from nltk.cluster.util import cosine_distance
import string
import math
import operator
from inltk.inltk import setup
import nltk
from nltk.tokenize import SpaceTokenizer
import re
from matplotlib.font_manager import FontProperties
hindi_font=FontProperties(fname = 'Nirmala-UI.ttf')
import indicnlp
from indicnlp.tokenize import sentence_tokenize
from inltk.inltk import get_sentence_similarity
from inltk.inltk import get_sentence_similarity
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df=pd.read_csv('train_hindi_new.csv',nrows=1000,delimiter=',')
pd.set_option('display.max_colwidth',None)

In [3]:
stopwords=pd.read_csv('StopwordsHindi.csv')
stopwords=list(stopwords['stopwords'])
stopwords.extend(['में','के','है','से'])

In [4]:
df.dropna(inplace=True)

In [5]:
df=df.head(500)

In [6]:
df.shape

(202, 3)

In [7]:
def remove_redundant_sentences(sentences):
    cleaned=[]
    for s in sentences:
        if s in cleaned or s.strip()=='':
            continue
        else:
            cleaned.append(s)
    return cleaned

def clean_corpus(corpus):
    corpus=corpus.replace('।','.')
    corpus=corpus.replace('\xa0','')
    corpus=corpus.replace('\n','')
    corpus=corpus.replace('\r','')
    return corpus

def get_clean_sentences(doc):
    cleaned_doc=clean_corpus(doc)
    sentences=cleaned_doc.split('.')
    sentences=remove_redundant_sentences(sentences)
    return sentences

In [8]:
patternEnglish = re.compile(r'[[A-Z]|[a-z]]*')
patternNumeric = re.compile(r'.*[0-9]+')
patternBraces=re.compile(r'\[(.*?)\]|\{(.*?)\}|\((.*?)\)')
patternUrl=re.compile(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))")

Method I : Rule based extractive summarization

In [9]:
def titlewordsScore(sentence,heading_titlewords):
    titlewords_sent=SpaceTokenizer().tokenize(sentence)
    titlewords_count=len(heading_titlewords)
    titlewords_sent=[word for word in titlewords_sent if word in heading_titlewords]
    s1_score=len(titlewords_sent)/titlewords_count
    return (s1_score)

In [10]:
def get_len_all_sentences(sentences):
    len_of_all_sent=[len(SpaceTokenizer().tokenize(sentence)) for sentence in sentences]
    max_length=np.max(len_of_all_sent)
    return max_length, len_of_all_sent

In [11]:
def searchEnglishWords(sentence):
    words=SpaceTokenizer().tokenize(sentence.strip())
    words=list(filter(None,words))
    english_words=[word for word in words if patternEnglish.match(word) !=None]
    return len(english_words)/len(words)

In [12]:
def positionScore(total_len):
    mid=(total_len/2)
    positions=(np.arange(total_len,mid,-1)/total_len).tolist()
    if(total_len%2==0):
        positions.extend((np.arange(mid+1,total_len+1,1)/total_len).tolist())
    else:
        positions.extend(np.arange(mid,total_len+1,1).tolist())
    return positions

In [13]:
def numericScore(sentence):
    words=SpaceTokenizer().tokenize(sentence)
    numbers=[1 for word in words if re.match(patternNumeric,word)]
    return np.sum(numbers)/len(words)

def bracesScore(sentence):
    t=re.findall(patternBraces,sentence)
    words=SpaceTokenizer().tokenize(sentence)
    insideTerms=()
    for term in t:
        insideTerms+=term
    insideTerms=[term for term in insideTerms if term !='']
    return (len(words)-len(insideTerms))/len(words)

In [14]:
def url_email_score(sentence):
    words=SpaceTokenizer().tokenize(sentence.strip())
    words=list(filter(None, words))
    urls=[word for word in words if patternUrl.match(word)!=None]
    return len(urls)/len(words)

In [15]:
def generate_summary_rule_based(clean_sentences,compression_ratio):
    score_table=pd.DataFrame(clean_sentences)
    score_table.columns=['sentence']

    headlinetokens=SpaceTokenizer().tokenize(sample['headline'])
    heading_titlewords=[ word for word in  headlinetokens if word!= '' and word not in stopwords]
    score_table['S1']=[titlewordsScore(sentence,heading_titlewords) for sentence in clean_sentences]
    max_length,len_of_all_sent=get_len_all_sentences(clean_sentences)
    score_table['S2']=[x/max_length for x in len_of_all_sent]
    score_table['S3']=[searchEnglishWords(sentence) for sentence in clean_sentences]
    score_table['S4']=[numericScore(sentence) for sentence in clean_sentences]
    score_table['S5']=[bracesScore(sentence) for sentence in clean_sentences]
    score_table['S7']=[url_email_score(sentence) for sentence in clean_sentences]
    #score_table['S8']=positionScore(len(clean_sentences))
    score_table['Total']=score_table['S1']+score_table['S2']+score_table['S3']+score_table['S4']+score_table['S5']+score_table['S7']
    summary_sent_count=round(compression_ratio*len(clean_sentences))
    summary_sent=list(score_table.sort_values('Total',ascending=False).iloc[0:summary_sent_count]['sentence'])
    summary=""
    for sentence in summary_sent:
        summary+=sentence.strip()+"| "
    return summary

In [16]:
sample=df.iloc[1]
clean_sentences=get_clean_sentences(sample['article'])
summary_rule_based=generate_summary_rule_based(clean_sentences,0.4)

print('Summary obtained from rule based strategy: ',summary_rule_based,'\n')


Summary obtained from rule based strategy:  अमेरिका में राष्ट्रपति पद के लिए डेमोक्रेटिक पार्टी की उम्मीदवार हिलेरी क्लिंटन निमोनिया से पीड़ित पाई गई हैं और उन्हें आराम करने की सलाह दी गई है| )चिकित्सक ने बताया कि हिलेरी को कल तड़के मैनहट्टन में ग्राउंड जीरो स्मारक में 9/11 हमलों के 15 साल पूरे होने के अवसर पर आयोजित एक स्मृति सभा में "बुखार हो गया और उनके शरीर में पानी की कमी हो गई| चिकित्सक ने बताया कि हिलेरी को कल तड़के मैनहट्टन में ग्राउंड जीरो स्मारक में 9/11 हमलों के 15 साल पूरे होने के अवसर पर आयोजित एक स्मृति सभा में "बुखार हो गया और उनके शरीर में पानी की कमी हो गई| )हिलेरी की प्रचार टीम ने बताया कि इस कार्यक्रम के दौरान वह अस्वस्थ हो गईं और अपनी बेटी चेल्सी के अपार्टमेंट के लिए रवाना हो गईं जो शहर के पॉश फ्लैटीरोन में है| टिप्पणियांहिलेरी की प्रचार टीम ने बताया कि इस कार्यक्रम के दौरान वह अस्वस्थ हो गईं और अपनी बेटी चेल्सी के अपार्टमेंट के लिए रवाना हो गईं जो शहर के पॉश फ्लैटीरोन में है| इससे कुछ ही घंटे पहले हिलेरी अस्वस्थ महसूस होने के कारण यहां 9/11 हमलों की स्मृति सभा से ब

In [17]:
summary=[]
for i in range(len(df)):
    clean_sentences=get_clean_sentences(sample['article'])
    summary_rule_based=generate_summary_rule_based(clean_sentences,0.4)
    summary.append(summary_rule_based)

In [18]:
df['rule_based']=summary_rule_based

In [19]:
from rouge import Rouge
rouge = Rouge()
score = rouge.get_scores(df['rule_based'], df['summary'], avg=True)
BartRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
BartRouge

,rouge-1,rouge-2,rouge-l
recall,0.239906,0.017990,0.203717
precision,0.064254,0.003556,0.054047
f-measure,0.100641,0.005887,0.084813


Text Summarization using TF-IDF scores of sentences¶

In [20]:
from nltk.tokenize import sent_tokenize
def create_frequency_table(sentences):
    word_freq={}
    for sentence in sentences:
        words=SpaceTokenizer().tokenize(sentence)
        for word in words:
            if word in stopwords:
                continue
            if word in word_freq:
                word_freq[word]+=1
            else:
                word_freq[word]=1
    return word_freq

In [21]:
def score_sentences(sentences,word_freq):
    sentenceValue={}
    for sentence in sentences:
        words_in_sentence=SpaceTokenizer().tokenize(sentence)
        word_count=len(words_in_sentence)
        for word in word_freq:
            if word in words_in_sentence:
                if sentence[:10] in sentenceValue:
                    sentenceValue[sentence[:10]]+=word_freq[word]
                else:
                    sentenceValue[sentence[:10]]=word_freq[word]
        sentenceValue[sentence[:10]]=sentenceValue[sentence[:10]]/word_count
    return sentenceValue
    

In [22]:
def average_sentence_value(sentenceValues):
    sum_values=0
    for sent_id in sentenceValues:
        sum_values+=sentenceValues[sent_id]
    sent_count=len(sentenceValues)
    return sum_values/sent_count

In [23]:
def generate_summary(avg_score,sentences,sentenceValue,identifier):
    sentence_count=0
    summary=''
    for sentence in sentences:
        if sentence[:identifier] in sentenceValue and sentenceValue[sentence[:identifier]]>=avg_score:
            summary+=sentence.strip()+"| "
            sentence_count+=1
    return summary,sentence_count

In [24]:
def summarise_term_frequency_sentence_weighing(sentences):
    word_freq_table=create_frequency_table(sentences)
    scores=score_sentences(sentences,word_freq_table)
    avg_score=average_sentence_value(scores)
    summary,sent_count=generate_summary(avg_score,sentences,scores,10)
    return summary

In [25]:
summary=[]
for i in range(len(df)):
    clean_sentences=get_clean_sentences(sample['article'])
    summary_term_frequency_sentence_weighing=summarise_term_frequency_sentence_weighing(clean_sentences)
    summary.append(summary_term_frequency_sentence_weighing)

In [26]:
df['summary_term_frequency_sentence_weighing']=summary

In [27]:
from rouge import Rouge
rouge = Rouge()
score = rouge.get_scores(df['summary_term_frequency_sentence_weighing'], df['summary'], avg=True)
BartRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
BartRouge

,rouge-1,rouge-2,rouge-l
recall,0.203657,0.004164,0.175468
precision,0.082379,0.001350,0.070390
f-measure,0.116275,0.002006,0.099600


In [28]:
def calc_term_frequency_sentence_wise(sentences):
    freq_matrix={}
    for sentence in sentences:
        words=SpaceTokenizer().tokenize(sentence)
        sent_freq_table={}
        for word in words:
            if word in stopwords:
                continue
            if word in sent_freq_table:
                sent_freq_table[word]+=1
            else:
                sent_freq_table[word]=1
        freq_matrix[sentence[:15]]=sent_freq_table
    return freq_matrix
        
def calc_tf_matrix(freq_matrix):
    for sentence,sent_freq_table in freq_matrix.items():
        tf_sent={}
        sent_len=len(sent_freq_table)
        for word in sent_freq_table:
            sent_freq_table[word]/=sent_len
    return freq_matrix

def calc_sentence_frequency(freq_matrix):
    sent_freq={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        for word,count in freq_matrix_sent.items():
            if word in sent_freq:
                sent_freq[word]+=1
            else:
                sent_freq[word]=1
    return sent_freq

def calc_idf_score(total_sentences,sent_freq,freq_matrix):
    idf={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        idf_sent={}
        for word in freq_matrix_sent:
            idf_sent[word]=math.log10(total_sentences / float(sent_freq[word]))
        idf[sentence]=idf_sent
    return idf
    
def calc_tf_idf_score(tf,idf):
    tf_idf={}
    for (sentence1,tf_sent),(sentence2,idf_sent) in zip(tf.items(),idf.items()):
        tf_idf_sent={}
        for (word1,tf_score),(word2,idf_score) in zip(tf_sent.items(),idf_sent.items()):
            tf_idf_sent[word1]=tf_score*idf_score
        tf_idf[sentence1]=tf_idf_sent
    return tf_idf

def calc_tf_idf_score_sentence_wise(tf_idf_matrix):
    tf_idf={}
    for sentence,tf_idf_sent in tf_idf_matrix.items():
        tf_idf_sentence=0
        for word,tf_idf_score in tf_idf_sent.items():
            tf_idf_sentence+=tf_idf_score
        tf_idf[sentence]=tf_idf_sentence
    return tf_idf
def get_tf_idf(sentences):
    sent_freq_matrix=calc_term_frequency_sentence_wise(sentences)
    freq_matrix=calc_tf_matrix(sent_freq_matrix)
    total_sentences=len(sentences)
    sent_freq=calc_sentence_frequency(freq_matrix)
    idf=calc_idf_score(total_sentences,sent_freq,freq_matrix)
    tf_idf_matrix=calc_tf_idf_score(freq_matrix,idf)
    tf_idf=calc_tf_idf_score_sentence_wise(tf_idf_matrix)
    return tf_idf

def summarise_tf_idf_sentence_weighting(sentences):
    tf_idf=get_tf_idf(sentences)
    sentences=remove_redundant_sentences(sentences)
    avg_tf_idf_score=average_sentence_value(tf_idf)
    tf_idf_summary,sent_count=generate_summary(avg_tf_idf_score,sentences,tf_idf,15)
    return tf_idf_summary

In [29]:
sample=df.iloc[1]
clean_sentences=def calc_term_frequency_sentence_wise(sentences):
    freq_matrix={}
    for sentence in sentences:
        words=SpaceTokenizer().tokenize(sentence)
        sent_freq_table={}
        for word in words:
            if word in stopwords:
                continue
            if word in sent_freq_table:
                sent_freq_table[word]+=1
            else:
                sent_freq_table[word]=1
        freq_matrix[sentence[:15]]=sent_freq_table
    return freq_matrix
        
def calc_tf_matrix(freq_matrix):
    for sentence,sent_freq_table in freq_matrix.items():
        tf_sent={}
        sent_len=len(sent_freq_table)
        for word in sent_freq_table:
            sent_freq_table[word]/=sent_len
    return freq_matrix

def calc_sentence_frequency(freq_matrix):
    sent_freq={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        for word,count in freq_matrix_sent.items():
            if word in sent_freq:
                sent_freq[word]+=1
            else:
                sent_freq[word]=1
    return sent_freq

def calc_idf_score(total_sentences,sent_freq,freq_matrix):
    idf={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        idf_sent={}
        for word in freq_matrix_sent:
            idf_sent[word]=math.log10(total_sentences / float(sent_freq[word]))
        idf[sentence]=idf_sent
    return idf
    
def calc_tf_idf_score(tf,idf):
    tf_idf={}
    for (sentence1,tf_sent),(sentence2,idf_sent) in zip(tf.items(),idf.items()):
        tf_idf_sent={}
        for (word1,tf_score),(word2,idf_score) in zip(tf_sent.items(),idf_sent.items()):
            tf_idf_sent[word1]=tf_score*idf_score
        tf_idf[sentence1]=tf_idf_sent
    return tf_idf

def calc_tf_idf_score_sentence_wise(tf_idf_matrix):
    tf_idf={}
    for sentence,tf_idf_sent in tf_idf_matrix.items():
        tf_idf_sentence=0
        for word,tf_idf_score in tf_idf_sent.items():
            tf_idf_sentence+=tf_idf_score
        tf_idf[sentence]=tf_idf_sentence
    return tf_idf
def get_tf_idf(sentences):
    sent_freq_matrix=calc_term_frequency_sentence_wise(sentences)
    freq_matrix=calc_tf_matrix(sent_freq_matrix)
    total_sentences=len(sentences)
    sent_freq=calc_sentence_frequency(freq_matrix)
    idf=calc_idf_score(total_sentences,sent_freq,freq_matrix)
    tf_idf_matrix=calc_tf_idf_score(freq_matrix,idf)
    tf_idf=calc_tf_idf_score_sentence_wise(tf_idf_matrix)
    return tf_idf

def summarise_tf_idf_sentence_weighting(sentences):
    tf_idf=get_tf_idf(sentences)
    sentences=remove_redundant_sentences(sentences)
    avg_tf_idf_score=average_sentence_value(tf_idf)
    tf_idf_summary,sent_count=generate_summary(avg_tf_idf_score,sentences,tf_idf,15)
    return tf_idf_summarydef calc_term_frequency_sentence_wise(sentences):
    freq_matrix={}
    for sentence in sentences:
        words=SpaceTokenizer().tokenize(sentence)
        sent_freq_table={}
        for word in words:
            if word in stopwords:
                continue
            if word in sent_freq_table:
                sent_freq_table[word]+=1
            else:
                sent_freq_table[word]=1
        freq_matrix[sentence[:15]]=sent_freq_table
    return freq_matrix
        
def calc_tf_matrix(freq_matrix):
    for sentence,sent_freq_table in freq_matrix.items():
        tf_sent={}
        sent_len=len(sent_freq_table)
        for word in sent_freq_table:
            sent_freq_table[word]/=sent_len
    return freq_matrix

def calc_sentence_frequency(freq_matrix):
    sent_freq={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        for word,count in freq_matrix_sent.items():
            if word in sent_freq:
                sent_freq[word]+=1
            else:
                sent_freq[word]=1
    return sent_freq

def calc_idf_score(total_sentences,sent_freq,freq_matrix):
    idf={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        idf_sent={}
        for word in freq_matrix_sent:
            idf_sent[word]=math.log10(total_sentences / float(sent_freq[word]))
        idf[sentence]=idf_sent
    return idf
    
def calc_tf_idf_score(tf,idf):
    tf_idf={}
    for (sentence1,tf_sent),(sentence2,idf_sent) in zip(tf.items(),idf.items()):
        tf_idf_sent={}
        for (word1,tf_score),(word2,idf_score) in zip(tf_sent.items(),idf_sent.items()):
            tf_idf_sent[word1]=tf_score*idf_score
        tf_idf[sentence1]=tf_idf_sent
    return tf_idf

def calc_tf_idf_score_sentence_wise(tf_idf_matrix):
    tf_idf={}
    for sentence,tf_idf_sent in tf_idf_matrix.items():
        tf_idf_sentence=0
        for word,tf_idf_score in tf_idf_sent.items():
            tf_idf_sentence+=tf_idf_score
        tf_idf[sentence]=tf_idf_sentence
    return tf_idf
def get_tf_idf(sentences):
    sent_freq_matrix=calc_term_frequency_sentence_wise(sentences)
    freq_matrix=calc_tf_matrix(sent_freq_matrix)
    total_sentences=len(sentences)
    sent_freq=calc_sentence_frequency(freq_matrix)
    idf=calc_idf_score(total_sentences,sent_freq,freq_matrix)
    tf_idf_matrix=calc_tf_idf_score(freq_matrix,idf)
    tf_idf=calc_tf_idf_score_sentence_wise(tf_idf_matrix)
    return tf_idf

def summarise_tf_idf_sentence_weighting(sentences):
    tf_idf=get_tf_idf(sentences)
    sentences=remove_redundant_sentences(sentences)
    avg_tf_idf_score=average_sentence_value(tf_idf)
    tf_idf_summary,sent_count=generate_summary(avg_tf_idf_score,sentences,tf_idf,15)
    return tf_idf_summarydef calc_term_frequency_sentence_wise(sentences):
    freq_matrix={}
    for sentence in sentences:
        words=SpaceTokenizer().tokenize(sentence)
        sent_freq_table={}
        for word in words:
            if word in stopwords:
                continue
            if word in sent_freq_table:
                sent_freq_table[word]+=1
            else:
                sent_freq_table[word]=1
        freq_matrix[sentence[:15]]=sent_freq_table
    return freq_matrix
        
def calc_tf_matrix(freq_matrix):
    for sentence,sent_freq_table in freq_matrix.items():
        tf_sent={}
        sent_len=len(sent_freq_table)
        for word in sent_freq_table:
            sent_freq_table[word]/=sent_len
    return freq_matrix

def calc_sentence_frequency(freq_matrix):
    sent_freq={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        for word,count in freq_matrix_sent.items():
            if word in sent_freq:
                sent_freq[word]+=1
            else:
                sent_freq[word]=1
    return sent_freq

def calc_idf_score(total_sentences,sent_freq,freq_matrix):
    idf={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        idf_sent={}
        for word in freq_matrix_sent:
            idf_sent[word]=math.log10(total_sentences / float(sent_freq[word]))
        idf[sentence]=idf_sent
    return idf
    
def calc_tf_idf_score(tf,idf):
    tf_idf={}
    for (sentence1,tf_sent),(sentence2,idf_sent) in zip(tf.items(),idf.items()):
        tf_idf_sent={}
        for (word1,tf_score),(word2,idf_score) in zip(tf_sent.items(),idf_sent.items()):
            tf_idf_sent[word1]=tf_score*idf_score
        tf_idf[sentence1]=tf_idf_sent
    return tf_idf

def calc_tf_idf_score_sentence_wise(tf_idf_matrix):
    tf_idf={}
    for sentence,tf_idf_sent in tf_idf_matrix.items():
        tf_idf_sentence=0
        for word,tf_idf_score in tf_idf_sent.items():
            tf_idf_sentence+=tf_idf_score
        tf_idf[sentence]=tf_idf_sentence
    return tf_idf
def get_tf_idf(sentences):
    sent_freq_matrix=calc_term_frequency_sentence_wise(sentences)
    freq_matrix=calc_tf_matrix(sent_freq_matrix)
    total_sentences=len(sentences)
    sent_freq=calc_sentence_frequency(freq_matrix)
    idf=calc_idf_score(total_sentences,sent_freq,freq_matrix)
    tf_idf_matrix=calc_tf_idf_score(freq_matrix,idf)
    tf_idf=calc_tf_idf_score_sentence_wise(tf_idf_matrix)
    return tf_idf

def summarise_tf_idf_sentence_weighting(sentences):
    tf_idf=get_tf_idf(sentences)
    sentences=remove_redundant_sentences(sentences)
    avg_tf_idf_score=average_sentence_value(tf_idf)
    tf_idf_summary,sent_count=generate_summary(avg_tf_idf_score,sentences,tf_idf,15)
    return tf_idf_summarydef calc_term_frequency_sentence_wise(sentences):
    freq_matrix={}
    for sentence in sentences:
        words=SpaceTokenizer().tokenize(sentence)
        sent_freq_table={}
        for word in words:
            if word in stopwords:
                continue
            if word in sent_freq_table:
                sent_freq_table[word]+=1
            else:
                sent_freq_table[word]=1
        freq_matrix[sentence[:15]]=sent_freq_table
    return freq_matrix
        
def calc_tf_matrix(freq_matrix):
    for sentence,sent_freq_table in freq_matrix.items():
        tf_sent={}
        sent_len=len(sent_freq_table)
        for word in sent_freq_table:
            sent_freq_table[word]/=sent_len
    return freq_matrix

def calc_sentence_frequency(freq_matrix):
    sent_freq={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        for word,count in freq_matrix_sent.items():
            if word in sent_freq:
                sent_freq[word]+=1
            else:
                sent_freq[word]=1
    return sent_freq

def calc_idf_score(total_sentences,sent_freq,freq_matrix):
    idf={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        idf_sent={}
        for word in freq_matrix_sent:
            idf_sent[word]=math.log10(total_sentences / float(sent_freq[word]))
        idf[sentence]=idf_sent
    return idf
    
def calc_tf_idf_score(tf,idf):
    tf_idf={}
    for (sentence1,tf_sent),(sentence2,idf_sent) in zip(tf.items(),idf.items()):
        tf_idf_sent={}
        for (word1,tf_score),(word2,idf_score) in zip(tf_sent.items(),idf_sent.items()):
            tf_idf_sent[word1]=tf_score*idf_score
        tf_idf[sentence1]=tf_idf_sent
    return tf_idf

def calc_tf_idf_score_sentence_wise(tf_idf_matrix):
    tf_idf={}
    for sentence,tf_idf_sent in tf_idf_matrix.items():
        tf_idf_sentence=0
        for word,tf_idf_score in tf_idf_sent.items():
            tf_idf_sentence+=tf_idf_score
        tf_idf[sentence]=tf_idf_sentence
    return tf_idf
def get_tf_idf(sentences):
    sent_freq_matrix=calc_term_frequency_sentence_wise(sentences)
    freq_matrix=calc_tf_matrix(sent_freq_matrix)
    total_sentences=len(sentences)
    sent_freq=calc_sentence_frequency(freq_matrix)
    idf=calc_idf_score(total_sentences,sent_freq,freq_matrix)
    tf_idf_matrix=calc_tf_idf_score(freq_matrix,idf)
    tf_idf=calc_tf_idf_score_sentence_wise(tf_idf_matrix)
    return tf_idf

def summarise_tf_idf_sentence_weighting(sentences):
    tf_idf=get_tf_idf(sentences)
    sentences=remove_redundant_sentences(sentences)
    avg_tf_idf_score=average_sentence_value(tf_idf)
    tf_idf_summary,sent_count=generate_summary(avg_tf_idf_score,sentences,tf_idf,15)
    return tf_idf_summarydef calc_term_frequency_sentence_wise(sentences):
    freq_matrix={}
    for sentence in sentences:
        words=SpaceTokenizer().tokenize(sentence)
        sent_freq_table={}
        for word in words:
            if word in stopwords:
                continue
            if word in sent_freq_table:
                sent_freq_table[word]+=1
            else:
                sent_freq_table[word]=1
        freq_matrix[sentence[:15]]=sent_freq_table
    return freq_matrix
        
def calc_tf_matrix(freq_matrix):
    for sentence,sent_freq_table in freq_matrix.items():
        tf_sent={}
        sent_len=len(sent_freq_table)
        for word in sent_freq_table:
            sent_freq_table[word]/=sent_len
    return freq_matrix

def calc_sentence_frequency(freq_matrix):
    sent_freq={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        for word,count in freq_matrix_sent.items():
            if word in sent_freq:
                sent_freq[word]+=1
            else:
                sent_freq[word]=1
    return sent_freq

def calc_idf_score(total_sentences,sent_freq,freq_matrix):
    idf={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        idf_sent={}
        for word in freq_matrix_sent:
            idf_sent[word]=math.log10(total_sentences / float(sent_freq[word]))
        idf[sentence]=idf_sent
    return idf
    
def calc_tf_idf_score(tf,idf):
    tf_idf={}
    for (sentence1,tf_sent),(sentence2,idf_sent) in zip(tf.items(),idf.items()):
        tf_idf_sent={}
        for (word1,tf_score),(word2,idf_score) in zip(tf_sent.items(),idf_sent.items()):
            tf_idf_sent[word1]=tf_score*idf_score
        tf_idf[sentence1]=tf_idf_sent
    return tf_idf

def calc_tf_idf_score_sentence_wise(tf_idf_matrix):
    tf_idf={}
    for sentence,tf_idf_sent in tf_idf_matrix.items():
        tf_idf_sentence=0
        for word,tf_idf_score in tf_idf_sent.items():
            tf_idf_sentence+=tf_idf_score
        tf_idf[sentence]=tf_idf_sentence
    return tf_idf
def get_tf_idf(sentences):
    sent_freq_matrix=calc_term_frequency_sentence_wise(sentences)
    freq_matrix=calc_tf_matrix(sent_freq_matrix)
    total_sentences=len(sentences)
    sent_freq=calc_sentence_frequency(freq_matrix)
    idf=calc_idf_score(total_sentences,sent_freq,freq_matrix)
    tf_idf_matrix=calc_tf_idf_score(freq_matrix,idf)
    tf_idf=calc_tf_idf_score_sentence_wise(tf_idf_matrix)
    return tf_idf

def summarise_tf_idf_sentence_weighting(sentences):
    tf_idf=get_tf_idf(sentences)
    sentences=remove_redundant_sentences(sentences)
    avg_tf_idf_score=average_sentence_value(tf_idf)
    tf_idf_summary,sent_count=generate_summary(avg_tf_idf_score,sentences,tf_idf,15)
    return tf_idf_summarydef calc_term_frequency_sentence_wise(sentences):
    freq_matrix={}
    for sentence in sentences:
        words=SpaceTokenizer().tokenize(sentence)
        sent_freq_table={}
        for word in words:
            if word in stopwords:
                continue
            if word in sent_freq_table:
                sent_freq_table[word]+=1
            else:
                sent_freq_table[word]=1
        freq_matrix[sentence[:15]]=sent_freq_table
    return freq_matrix
        
def calc_tf_matrix(freq_matrix):
    for sentence,sent_freq_table in freq_matrix.items():
        tf_sent={}
        sent_len=len(sent_freq_table)
        for word in sent_freq_table:
            sent_freq_table[word]/=sent_len
    return freq_matrix

def calc_sentence_frequency(freq_matrix):
    sent_freq={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        for word,count in freq_matrix_sent.items():
            if word in sent_freq:
                sent_freq[word]+=1
            else:
                sent_freq[word]=1
    return sent_freq

def calc_idf_score(total_sentences,sent_freq,freq_matrix):
    idf={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        idf_sent={}
        for word in freq_matrix_sent:
            idf_sent[word]=math.log10(total_sentences / float(sent_freq[word]))
        idf[sentence]=idf_sent
    return idf
    
def calc_tf_idf_score(tf,idf):
    tf_idf={}
    for (sentence1,tf_sent),(sentence2,idf_sent) in zip(tf.items(),idf.items()):
        tf_idf_sent={}
        for (word1,tf_score),(word2,idf_score) in zip(tf_sent.items(),idf_sent.items()):
            tf_idf_sent[word1]=tf_score*idf_score
        tf_idf[sentence1]=tf_idf_sent
    return tf_idf

def calc_tf_idf_score_sentence_wise(tf_idf_matrix):
    tf_idf={}
    for sentence,tf_idf_sent in tf_idf_matrix.items():
        tf_idf_sentence=0
        for word,tf_idf_score in tf_idf_sent.items():
            tf_idf_sentence+=tf_idf_score
        tf_idf[sentence]=tf_idf_sentence
    return tf_idf
def get_tf_idf(sentences):
    sent_freq_matrix=calc_term_frequency_sentence_wise(sentences)
    freq_matrix=calc_tf_matrix(sent_freq_matrix)
    total_sentences=len(sentences)
    sent_freq=calc_sentence_frequency(freq_matrix)
    idf=calc_idf_score(total_sentences,sent_freq,freq_matrix)
    tf_idf_matrix=calc_tf_idf_score(freq_matrix,idf)
    tf_idf=calc_tf_idf_score_sentence_wise(tf_idf_matrix)
    return tf_idf

def summarise_tf_idf_sentence_weighting(sentences):
    tf_idf=get_tf_idf(sentences)
    sentences=remove_redundant_sentences(sentences)
    avg_tf_idf_score=average_sentence_value(tf_idf)
    tf_idf_summary,sent_count=generate_summary(avg_tf_idf_score,sentences,tf_idf,15)
    return tf_idf_summarydef calc_term_frequency_sentence_wise(sentences):
    freq_matrix={}
    for sentence in sentences:
        words=SpaceTokenizer().tokenize(sentence)
        sent_freq_table={}
        for word in words:
            if word in stopwords:
                continue
            if word in sent_freq_table:
                sent_freq_table[word]+=1
            else:
                sent_freq_table[word]=1
        freq_matrix[sentence[:15]]=sent_freq_table
    return freq_matrix
        
def calc_tf_matrix(freq_matrix):
    for sentence,sent_freq_table in freq_matrix.items():
        tf_sent={}
        sent_len=len(sent_freq_table)
        for word in sent_freq_table:
            sent_freq_table[word]/=sent_len
    return freq_matrix

def calc_sentence_frequency(freq_matrix):
    sent_freq={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        for word,count in freq_matrix_sent.items():
            if word in sent_freq:
                sent_freq[word]+=1
            else:
                sent_freq[word]=1
    return sent_freq

def calc_idf_score(total_sentences,sent_freq,freq_matrix):
    idf={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        idf_sent={}
        for word in freq_matrix_sent:
            idf_sent[word]=math.log10(total_sentences / float(sent_freq[word]))
        idf[sentence]=idf_sent
    return idf
    
def calc_tf_idf_score(tf,idf):
    tf_idf={}
    for (sentence1,tf_sent),(sentence2,idf_sent) in zip(tf.items(),idf.items()):
        tf_idf_sent={}
        for (word1,tf_score),(word2,idf_score) in zip(tf_sent.items(),idf_sent.items()):
            tf_idf_sent[word1]=tf_score*idf_score
        tf_idf[sentence1]=tf_idf_sent
    return tf_idf

def calc_tf_idf_score_sentence_wise(tf_idf_matrix):
    tf_idf={}
    for sentence,tf_idf_sent in tf_idf_matrix.items():
        tf_idf_sentence=0
        for word,tf_idf_score in tf_idf_sent.items():
            tf_idf_sentence+=tf_idf_score
        tf_idf[sentence]=tf_idf_sentence
    return tf_idf
def get_tf_idf(sentences):
    sent_freq_matrix=calc_term_frequency_sentence_wise(sentences)
    freq_matrix=calc_tf_matrix(sent_freq_matrix)
    total_sentences=len(sentences)
    sent_freq=calc_sentence_frequency(freq_matrix)
    idf=calc_idf_score(total_sentences,sent_freq,freq_matrix)
    tf_idf_matrix=calc_tf_idf_score(freq_matrix,idf)
    tf_idf=calc_tf_idf_score_sentence_wise(tf_idf_matrix)
    return tf_idf

def summarise_tf_idf_sentence_weighting(sentences):
    tf_idf=get_tf_idf(sentences)
    sentences=remove_redundant_sentences(sentences)
    avg_tf_idf_score=average_sentence_value(tf_idf)
    tf_idf_summary,sent_count=generate_summary(avg_tf_idf_score,sentences,tf_idf,15)
    return tf_idf_summarydef calc_term_frequency_sentence_wise(sentences):
    freq_matrix={}
    for sentence in sentences:
        words=SpaceTokenizer().tokenize(sentence)
        sent_freq_table={}
        for word in words:
            if word in stopwords:
                continue
            if word in sent_freq_table:
                sent_freq_table[word]+=1
            else:
                sent_freq_table[word]=1
        freq_matrix[sentence[:15]]=sent_freq_table
    return freq_matrix
        
def calc_tf_matrix(freq_matrix):
    for sentence,sent_freq_table in freq_matrix.items():
        tf_sent={}
        sent_len=len(sent_freq_table)
        for word in sent_freq_table:
            sent_freq_table[word]/=sent_len
    return freq_matrix

def calc_sentence_frequency(freq_matrix):
    sent_freq={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        for word,count in freq_matrix_sent.items():
            if word in sent_freq:
                sent_freq[word]+=1
            else:
                sent_freq[word]=1
    return sent_freq

def calc_idf_score(total_sentences,sent_freq,freq_matrix):
    idf={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        idf_sent={}
        for word in freq_matrix_sent:
            idf_sent[word]=math.log10(total_sentences / float(sent_freq[word]))
        idf[sentence]=idf_sent
    return idf
    
def calc_tf_idf_score(tf,idf):
    tf_idf={}
    for (sentence1,tf_sent),(sentence2,idf_sent) in zip(tf.items(),idf.items()):
        tf_idf_sent={}
        for (word1,tf_score),(word2,idf_score) in zip(tf_sent.items(),idf_sent.items()):
            tf_idf_sent[word1]=tf_score*idf_score
        tf_idf[sentence1]=tf_idf_sent
    return tf_idf

def calc_tf_idf_score_sentence_wise(tf_idf_matrix):
    tf_idf={}
    for sentence,tf_idf_sent in tf_idf_matrix.items():
        tf_idf_sentence=0
        for word,tf_idf_score in tf_idf_sent.items():
            tf_idf_sentence+=tf_idf_score
        tf_idf[sentence]=tf_idf_sentence
    return tf_idf
def get_tf_idf(sentences):
    sent_freq_matrix=calc_term_frequency_sentence_wise(sentences)
    freq_matrix=calc_tf_matrix(sent_freq_matrix)
    total_sentences=len(sentences)
    sent_freq=calc_sentence_frequency(freq_matrix)
    idf=calc_idf_score(total_sentences,sent_freq,freq_matrix)
    tf_idf_matrix=calc_tf_idf_score(freq_matrix,idf)
    tf_idf=calc_tf_idf_score_sentence_wise(tf_idf_matrix)
    return tf_idf

def summarise_tf_idf_sentence_weighting(sentences):
    tf_idf=get_tf_idf(sentences)
    sentences=remove_redundant_sentences(sentences)
    avg_tf_idf_score=average_sentence_value(tf_idf)
    tf_idf_summary,sent_count=generate_summary(avg_tf_idf_score,sentences,tf_idf,15)
    return tf_idf_summarydef calc_term_frequency_sentence_wise(sentences):
    freq_matrix={}
    for sentence in sentences:
        words=SpaceTokenizer().tokenize(sentence)
        sent_freq_table={}
        for word in words:
            if word in stopwords:
                continue
            if word in sent_freq_table:
                sent_freq_table[word]+=1
            else:
                sent_freq_table[word]=1
        freq_matrix[sentence[:15]]=sent_freq_table
    return freq_matrix
        
def calc_tf_matrix(freq_matrix):
    for sentence,sent_freq_table in freq_matrix.items():
        tf_sent={}
        sent_len=len(sent_freq_table)
        for word in sent_freq_table:
            sent_freq_table[word]/=sent_len
    return freq_matrix

def calc_sentence_frequency(freq_matrix):
    sent_freq={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        for word,count in freq_matrix_sent.items():
            if word in sent_freq:
                sent_freq[word]+=1
            else:
                sent_freq[word]=1
    return sent_freq

def calc_idf_score(total_sentences,sent_freq,freq_matrix):
    idf={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        idf_sent={}
        for word in freq_matrix_sent:
            idf_sent[word]=math.log10(total_sentences / float(sent_freq[word]))
        idf[sentence]=idf_sent
    return idf
    
def calc_tf_idf_score(tf,idf):
    tf_idf={}
    for (sentence1,tf_sent),(sentence2,idf_sent) in zip(tf.items(),idf.items()):
        tf_idf_sent={}
        for (word1,tf_score),(word2,idf_score) in zip(tf_sent.items(),idf_sent.items()):
            tf_idf_sent[word1]=tf_score*idf_score
        tf_idf[sentence1]=tf_idf_sent
    return tf_idf

def calc_tf_idf_score_sentence_wise(tf_idf_matrix):
    tf_idf={}
    for sentence,tf_idf_sent in tf_idf_matrix.items():
        tf_idf_sentence=0
        for word,tf_idf_score in tf_idf_sent.items():
            tf_idf_sentence+=tf_idf_score
        tf_idf[sentence]=tf_idf_sentence
    return tf_idf
def get_tf_idf(sentences):
    sent_freq_matrix=calc_term_frequency_sentence_wise(sentences)
    freq_matrix=calc_tf_matrix(sent_freq_matrix)
    total_sentences=len(sentences)
    sent_freq=calc_sentence_frequency(freq_matrix)
    idf=calc_idf_score(total_sentences,sent_freq,freq_matrix)
    tf_idf_matrix=calc_tf_idf_score(freq_matrix,idf)
    tf_idf=calc_tf_idf_score_sentence_wise(tf_idf_matrix)
    return tf_idf

def summarise_tf_idf_sentence_weighting(sentences):
    tf_idf=get_tf_idf(sentences)
    sentences=remove_redundant_sentences(sentences)
    avg_tf_idf_score=average_sentence_value(tf_idf)
    tf_idf_summary,sent_count=generate_summary(avg_tf_idf_score,sentences,tf_idf,15)
    return tf_idf_summarydef calc_term_frequency_sentence_wise(sentences):
    freq_matrix={}
    for sentence in sentences:
        words=SpaceTokenizer().tokenize(sentence)
        sent_freq_table={}
        for word in words:
            if word in stopwords:
                continue
            if word in sent_freq_table:
                sent_freq_table[word]+=1
            else:
                sent_freq_table[word]=1
        freq_matrix[sentence[:15]]=sent_freq_table
    return freq_matrix
        
def calc_tf_matrix(freq_matrix):
    for sentence,sent_freq_table in freq_matrix.items():
        tf_sent={}
        sent_len=len(sent_freq_table)
        for word in sent_freq_table:
            sent_freq_table[word]/=sent_len
    return freq_matrix

def calc_sentence_frequency(freq_matrix):
    sent_freq={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        for word,count in freq_matrix_sent.items():
            if word in sent_freq:
                sent_freq[word]+=1
            else:
                sent_freq[word]=1
    return sent_freq

def calc_idf_score(total_sentences,sent_freq,freq_matrix):
    idf={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        idf_sent={}
        for word in freq_matrix_sent:
            idf_sent[word]=math.log10(total_sentences / float(sent_freq[word]))
        idf[sentence]=idf_sent
    return idf
    
def calc_tf_idf_score(tf,idf):
    tf_idf={}
    for (sentence1,tf_sent),(sentence2,idf_sent) in zip(tf.items(),idf.items()):
        tf_idf_sent={}
        for (word1,tf_score),(word2,idf_score) in zip(tf_sent.items(),idf_sent.items()):
            tf_idf_sent[word1]=tf_score*idf_score
        tf_idf[sentence1]=tf_idf_sent
    return tf_idf

def calc_tf_idf_score_sentence_wise(tf_idf_matrix):
    tf_idf={}
    for sentence,tf_idf_sent in tf_idf_matrix.items():
        tf_idf_sentence=0
        for word,tf_idf_score in tf_idf_sent.items():
            tf_idf_sentence+=tf_idf_score
        tf_idf[sentence]=tf_idf_sentence
    return tf_idf
def get_tf_idf(sentences):
    sent_freq_matrix=calc_term_frequency_sentence_wise(sentences)
    freq_matrix=calc_tf_matrix(sent_freq_matrix)
    total_sentences=len(sentences)
    sent_freq=calc_sentence_frequency(freq_matrix)
    idf=calc_idf_score(total_sentences,sent_freq,freq_matrix)
    tf_idf_matrix=calc_tf_idf_score(freq_matrix,idf)
    tf_idf=calc_tf_idf_score_sentence_wise(tf_idf_matrix)
    return tf_idf

def summarise_tf_idf_sentence_weighting(sentences):
    tf_idf=get_tf_idf(sentences)
    sentences=remove_redundant_sentences(sentences)
    avg_tf_idf_score=average_sentence_value(tf_idf)
    tf_idf_summary,sent_count=generate_summary(avg_tf_idf_score,sentences,tf_idf,15)
    return tf_idf_summarydef calc_term_frequency_sentence_wise(sentences):
    freq_matrix={}
    for sentence in sentences:
        words=SpaceTokenizer().tokenize(sentence)
        sent_freq_table={}
        for word in words:
            if word in stopwords:
                continue
            if word in sent_freq_table:
                sent_freq_table[word]+=1
            else:
                sent_freq_table[word]=1
        freq_matrix[sentence[:15]]=sent_freq_table
    return freq_matrix
        
def calc_tf_matrix(freq_matrix):
    for sentence,sent_freq_table in freq_matrix.items():
        tf_sent={}
        sent_len=len(sent_freq_table)
        for word in sent_freq_table:
            sent_freq_table[word]/=sent_len
    return freq_matrix

def calc_sentence_frequency(freq_matrix):
    sent_freq={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        for word,count in freq_matrix_sent.items():
            if word in sent_freq:
                sent_freq[word]+=1
            else:
                sent_freq[word]=1
    return sent_freq

def calc_idf_score(total_sentences,sent_freq,freq_matrix):
    idf={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        idf_sent={}
        for word in freq_matrix_sent:
            idf_sent[word]=math.log10(total_sentences / float(sent_freq[word]))
        idf[sentence]=idf_sent
    return idf
    
def calc_tf_idf_score(tf,idf):
    tf_idf={}
    for (sentence1,tf_sent),(sentence2,idf_sent) in zip(tf.items(),idf.items()):
        tf_idf_sent={}
        for (word1,tf_score),(word2,idf_score) in zip(tf_sent.items(),idf_sent.items()):
            tf_idf_sent[word1]=tf_score*idf_score
        tf_idf[sentence1]=tf_idf_sent
    return tf_idf

def calc_tf_idf_score_sentence_wise(tf_idf_matrix):
    tf_idf={}
    for sentence,tf_idf_sent in tf_idf_matrix.items():
        tf_idf_sentence=0
        for word,tf_idf_score in tf_idf_sent.items():
            tf_idf_sentence+=tf_idf_score
        tf_idf[sentence]=tf_idf_sentence
    return tf_idf
def get_tf_idf(sentences):
    sent_freq_matrix=calc_term_frequency_sentence_wise(sentences)
    freq_matrix=calc_tf_matrix(sent_freq_matrix)
    total_sentences=len(sentences)
    sent_freq=calc_sentence_frequency(freq_matrix)
    idf=calc_idf_score(total_sentences,sent_freq,freq_matrix)
    tf_idf_matrix=calc_tf_idf_score(freq_matrix,idf)
    tf_idf=calc_tf_idf_score_sentence_wise(tf_idf_matrix)
    return tf_idf

def summarise_tf_idf_sentence_weighting(sentences):
    tf_idf=get_tf_idf(sentences)
    sentences=remove_redundant_sentences(sentences)
    avg_tf_idf_score=average_sentence_value(tf_idf)
    tf_idf_summary,sent_count=generate_summary(avg_tf_idf_score,sentences,tf_idf,15)
    return tf_idf_summarydef calc_term_frequency_sentence_wise(sentences):
    freq_matrix={}
    for sentence in sentences:
        words=SpaceTokenizer().tokenize(sentence)
        sent_freq_table={}
        for word in words:
            if word in stopwords:
                continue
            if word in sent_freq_table:
                sent_freq_table[word]+=1
            else:
                sent_freq_table[word]=1
        freq_matrix[sentence[:15]]=sent_freq_table
    return freq_matrix
        
def calc_tf_matrix(freq_matrix):
    for sentence,sent_freq_table in freq_matrix.items():
        tf_sent={}
        sent_len=len(sent_freq_table)
        for word in sent_freq_table:
            sent_freq_table[word]/=sent_len
    return freq_matrix

def calc_sentence_frequency(freq_matrix):
    sent_freq={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        for word,count in freq_matrix_sent.items():
            if word in sent_freq:
                sent_freq[word]+=1
            else:
                sent_freq[word]=1
    return sent_freq

def calc_idf_score(total_sentences,sent_freq,freq_matrix):
    idf={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        idf_sent={}
        for word in freq_matrix_sent:
            idf_sent[word]=math.log10(total_sentences / float(sent_freq[word]))
        idf[sentence]=idf_sent
    return idf
    
def calc_tf_idf_score(tf,idf):
    tf_idf={}
    for (sentence1,tf_sent),(sentence2,idf_sent) in zip(tf.items(),idf.items()):
        tf_idf_sent={}
        for (word1,tf_score),(word2,idf_score) in zip(tf_sent.items(),idf_sent.items()):
            tf_idf_sent[word1]=tf_score*idf_score
        tf_idf[sentence1]=tf_idf_sent
    return tf_idf

def calc_tf_idf_score_sentence_wise(tf_idf_matrix):
    tf_idf={}
    for sentence,tf_idf_sent in tf_idf_matrix.items():
        tf_idf_sentence=0
        for word,tf_idf_score in tf_idf_sent.items():
            tf_idf_sentence+=tf_idf_score
        tf_idf[sentence]=tf_idf_sentence
    return tf_idf
def get_tf_idf(sentences):
    sent_freq_matrix=calc_term_frequency_sentence_wise(sentences)
    freq_matrix=calc_tf_matrix(sent_freq_matrix)
    total_sentences=len(sentences)
    sent_freq=calc_sentence_frequency(freq_matrix)
    idf=calc_idf_score(total_sentences,sent_freq,freq_matrix)
    tf_idf_matrix=calc_tf_idf_score(freq_matrix,idf)
    tf_idf=calc_tf_idf_score_sentence_wise(tf_idf_matrix)
    return tf_idf

def summarise_tf_idf_sentence_weighting(sentences):
    tf_idf=get_tf_idf(sentences)
    sentences=remove_redundant_sentences(sentences)
    avg_tf_idf_score=average_sentence_value(tf_idf)
    tf_idf_summary,sent_count=generate_summary(avg_tf_idf_score,sentences,tf_idf,15)
    return tf_idf_summaryget_clean_sentences(sample['article'])
summary_tf_idf=summarise_tf_idf_sentence_weighting(clean_sentences)

print('Summary obtained from rule based strategy: ',summary_rule_based,'\n')

SyntaxError: invalid syntax (1314056363.py, line 2)

In [56]:
clean=[]
for i in df['article']:
    clean.append(get_clean_sentences(i))
tf_idf_summary=[]
for j in clean:
    tf_idf_summary.append(summarise_tf_idf_sentence_weighting(j))

In [57]:
df['tf_idf_summary']=tf_idf_summary

In [58]:
df.head()

headline  \
3                                                       रिम करेगा दो हजार कर्मचारियों की छंटनी     
7   अमेरिका : राष्ट्रपति पद की उम्मीदवार हिलेरी क्लिंटन को हुआ निमोनिया, डॉक्टरों ने की पुष्टि     
8                            देश के सर्वश्रेष्ठ स्पिनरों में से एक है हरभजन सिंह : अनिल कुंबले     
9            इंडिगो के स्टाफ ने दिल्ली एयरपोर्ट पर पैसेंजर से की मारपीट, एयरलाइन ने माफी मांगी     
10              नव्या नवेली के साथ दोस्ती पर जावेद जाफरी के बेटे ने किया खुलासा, कही यह बात...     

                                                                                                                                                                                                          summary  \
3                                               ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रिसर्च इन मोशन (रिम) अपनी पुनर्गठन योजना के तहत अगले कुछ सप्ताह में वैश्विक स्तर पर 2,000 कर्मचारियों की छंटनी करेगी।   
7                             श्रद्धांजलि सभा के दौरान अचानक खराब हुई हिलेरी क्लिंटन की तबीयत.\r\nडॉक्टरों ने हिलेरी को निमोनिया होने की पुष्टि की है.\r\nराष्ट्रपति पद के लिए डेमोक्रेटिक उम्मीदवार हैं क्लिंटन.   
8   पूर्व भारतीय कप्तान अनिल कुंबले का मानना है कि भारत का स्पिन विभाग स्तरीय गेंदबाजों से भरा है, और उन्होंने टीम में वापसी करने वाले ऑफ-स्पिनर हरभजन सिंह को 'देश के सर्वश्रेष्ठ स्पिनरों' में से एक करार दिया।   
9                                                                               इंडिगो के ग्राउंड स्टाफ ने यात्री से की हाथापाई\r\nइंडिगो एयरलाइन ने इस घटना के लिए माफी मांग ली है.\r\n15 अक्टूबर की है ये घटना.   
10                                                                                         मीजान जाफरी ने किया खुलासा\r\nनव्या नंदा को बताया अच्छा दोस्त\r\nफिल्म 'मलाल' से बॉलीवुड में एंट्री करेंगे मीजान जाफरी   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [69]:
from rouge import Rouge
rouge = Rouge()
score = rouge.get_scores(df['tf_idf_summary'], df['summary'], avg=True)
Tf_idf = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
Tf_idf

,rouge-1,rouge-2,rouge-l
recall,0.645501,0.397652,0.530483
precision,0.214817,0.106975,0.180209
f-measure,0.297218,0.155229,0.247778


In [67]:
df.head()

headline  \
3                                                       रिम करेगा दो हजार कर्मचारियों की छंटनी     
7   अमेरिका : राष्ट्रपति पद की उम्मीदवार हिलेरी क्लिंटन को हुआ निमोनिया, डॉक्टरों ने की पुष्टि     
8                            देश के सर्वश्रेष्ठ स्पिनरों में से एक है हरभजन सिंह : अनिल कुंबले     
9            इंडिगो के स्टाफ ने दिल्ली एयरपोर्ट पर पैसेंजर से की मारपीट, एयरलाइन ने माफी मांगी     
10              नव्या नवेली के साथ दोस्ती पर जावेद जाफरी के बेटे ने किया खुलासा, कही यह बात...     

                                                                                                                                                                                                          summary  \
3                                               ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रिसर्च इन मोशन (रिम) अपनी पुनर्गठन योजना के तहत अगले कुछ सप्ताह में वैश्विक स्तर पर 2,000 कर्मचारियों की छंटनी करेगी।   
7                             श्रद्धांजलि सभा के दौरान अचानक खराब हुई हिलेरी क्लिंटन की तबीयत.\r\nडॉक्टरों ने हिलेरी को निमोनिया होने की पुष्टि की है.\r\nराष्ट्रपति पद के लिए डेमोक्रेटिक उम्मीदवार हैं क्लिंटन.   
8   पूर्व भारतीय कप्तान अनिल कुंबले का मानना है कि भारत का स्पिन विभाग स्तरीय गेंदबाजों से भरा है, और उन्होंने टीम में वापसी करने वाले ऑफ-स्पिनर हरभजन सिंह को 'देश के सर्वश्रेष्ठ स्पिनरों' में से एक करार दिया।   
9                                                                               इंडिगो के ग्राउंड स्टाफ ने यात्री से की हाथापाई\r\nइंडिगो एयरलाइन ने इस घटना के लिए माफी मांग ली है.\r\n15 अक्टूबर की है ये घटना.   
10                                                                                         मीजान जाफरी ने किया खुलासा\r\nनव्या नंदा को बताया अच्छा दोस्त\r\nफिल्म 'मलाल' से बॉलीवुड में एंट्री करेंगे मीजान जाफरी   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [68]:
import plotly.express as px
import matplotlib.pyplot as plt

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)